In [3]:
from typing import List, Dict, Any
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_structured_chat_agent, create_react_agent
from langchain.tools import Tool
from langchain_core.tools import BaseTool
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
import os 
import json
from dotenv import load_dotenv

In [4]:
load_dotenv('../../.env')

model=os.environ.get('model')
print(model)

os.environ["OPENAI_API_VERSION"] = os.environ.get('AZURE_API_VERSION')
os.environ["AZURE_OPENAI_ENDPOINT"] = os.environ.get('AZURE_API_BASE')
os.environ["AZURE_OPENAI_API_KEY"] = os.environ.get('AZURE_API_KEY')

azure/gpt-4


In [5]:
llm = AzureChatOpenAI(
    azure_deployment="gpt-4",  # or your deployment
    api_version=os.environ.get('AZURE_API_VERSION'),
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# Example for create_react_agent

In [6]:
# Define custom tools
def search_database(query: str) -> str:
    """Search the database for information."""
    print('search_database invoked: '+ query)
    # Mock implementation
    return f"Found results for: {query}"

def update_record(input_str: str) -> str:
    """Update a record in the database."""
    print('update_record invoked: '+ input_str)
    # Mock implementation
    return f"Updated record: {input_str}"

In [7]:
# Create Tool instances
tools = [
    Tool(
        name="database_search",
        func=search_database,
        description="Useful for searching information in the database. Input should be a search query."
    ),
    Tool(
        name="update_record",
        func=update_record,
        description="Useful for updating records in the database. Input should be the record details to update."
    )
]    

In [8]:
# Create prompt template with agent_scratchpad
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful AI assistant that helps users with database operations.
    You have access to the following tools:
    
    {tools}
    
    Use the following format:
    
    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question
    
    Begin!
    
    Question: {input}
    {agent_scratchpad}""")
])

In [9]:
# Create the agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=3,
    return_intermediate_steps=True
)

In [10]:
queries = [
    "Search for customer with ID 12345",
    "Update John Doe's email to john.doe@example.com"
]

for query in queries:
    print(f"\nQuery: {query}")
    result = agent_executor.invoke({"input": query})  ## For asynchrnous invocation - use ainvoke method
    print(f"Result: {result}")


Query: Search for customer with ID 12345


> Entering new AgentExecutor chain...
Thought: I need to search the database for the customer with the specified ID.
Action: database_search
Action Input: "customer ID 12345"
search_database invoked: customer ID 12345"

Found results for: customer ID 12345"
I now know the final answer

Final Answer: The customer with ID 12345 has been found in the database.

> Finished chain.
Result: {'input': 'Search for customer with ID 12345', 'output': 'The customer with ID 12345 has been found in the database.', 'intermediate_steps': [(AgentAction(tool='database_search', tool_input='customer ID 12345"\n', log='Thought: I need to search the database for the customer with the specified ID.\nAction: database_search\nAction Input: "customer ID 12345"\n'), 'Found results for: customer ID 12345"\n')]}

Query: Update John Doe's email to john.doe@example.com


> Entering new AgentExecutor chain...
Thought: To update John Doe's email, I need to first find the cur

# Example for create_structured_chat_agent

In [23]:
class TravelTools:
    @staticmethod
    def search_flights():
        return Tool(
            name="search_flights",
            func=lambda x: "Final Answer: Flight search results for: " + x,
            description="Search for available flights"
        )

In [29]:
tools = [TravelTools.search_flights()]
tools

tool_names = ["search_flights"]

In [31]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

custom_prompt = ChatPromptTemplate.from_template(
    '''Respond to the human as helpfully and accurately as possible. 
    
    You have access to the following tools: {tools}

    Valid "action" values: "Final Answer" or {tool_names}
 
    To use a tool, you MUST respond with a JSON object in the following format:
    {{ "action": "tool_name", "action_input": "tool_input" }}

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
Observation: action result
... (repeat Thought/Action/Observation N times)
We action result contains the phrase 'Final Answer', do not repeat further.

Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}

Human: {input}
Assistant: 

{agent_scratchpad}

'''
)

In [32]:
agent = create_structured_chat_agent(llm, tools, custom_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose =True,handle_parsing_errors=True)

result = agent_executor.invoke({"input": "I need to book a flight from New York to London", "tool_names": tool_names})
result



> Entering new AgentExecutor chain...
Thought: The user is requesting help to find a flight, which means I should use the search_flights tool to provide them with options.

Action:
```json
{
  "action": "search_flights",
  "action_input": {"tool_input": "New York to London"}
}
Thought: Since the observation indicates a "Final Answer" with flight search results, I do not need to take any further actions.

Action:
```
{
  "action": "Final Answer",
  "action_input": "Flight search results for: New York to London"
}
```

> Finished chain.


{'input': 'I need to book a flight from New York to London',
 'tool_names': ['search_flights'],
 'output': 'Flight search results for: New York to London'}